# Attacking random with RNN

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy # Add Deepcopy for args
import seaborn as sns 
import matplotlib.pyplot as plt

## Preparing Data
Input : 100 sequential random integer from java.util.Random.nextInt(1024)

Output : next data right after such generation

data set : 70000

train : 50000

validation : 10000

java.util.Random has following nextInt creation

```
def next(self, bits):
    if bits < 1:
        bits = 1
    elif bits > 32:
        bits = 32

    self._seed = (self._seed * 0x5deece66d + 0xb) & ((1 << 48) - 1)
    retval = self._seed >> (48 - bits)

    return retval
```



In [29]:
import random
import sys
import numpy as np

_seed = "aiming".__hash__()

def next(): # 32 bit integer
    global _seed
    _seed = (_seed * 0x5deece66d + 0xb) & ((1 << 48) - 1) 
    return _seed >> 38

x = []
y = []
for i in range(60000):
    temp = []
    for i in range(20):
        ar = np.zeros(1024, dtype = np.float16)
        ar[next()] = 1
        temp.append(ar)
    x.append(temp)
    ar = np.zeros(1024, dtype = np.float16)
    ar[next()] = 1
    y.append(ar)

npar_x = np.array(x)
npar_y = np.array(y)
torch_x = torch.from_numpy(npar_x)
torch_y = torch.from_numpy(npar_y)
print(npar_x.shape, npar_y.shape)

(60000, 20, 1024) (60000, 1024)


In [0]:
my_dataset = torch.utils.data.TensorDataset(torch_x,torch_y)
trainset, valset = torch.utils.data.random_split(my_dataset, [50000, 10000])
partition = {'train': trainset, 'val':valset}

## Model Construction

RNN model that accepts various models, we use MLP for each h2h, x2h, h2y.

In [0]:
class RNN(nn.Module):
    def __init__(self, batch_size, act, h2h, x2h, h2y):
        
        self.in_dim = 1024
        self.hid_dim = hid_dim
        self.out_dim = 1024
        self.batch_size = batch_size
        self.act = act
        
        self.h2h = h2h
        self.x2h = x2h
        self.h2y = h2y
        
        if self.act == "relu": 
            self.act_fn = nn.ReLU()
        elif self.act == "tanh": 
            self.act_fn = nn.Tanh()
        elif self.act == "sigmoid": 
            self.act_fn = nn.sigmoid()
        else: 
            raise ValueError("Illegal activation function")
        
        self.hidden = self.init_hidden(self.batch_size)
        
    def init_hidden(self, batch_size = None):
        if batch_size is None: batch_size = self.batch_size
        return torch.zeros(batch_size, self.hid_dim)

    def forward(self, x):
        h = self.act_fn(self.h2h(self.hidden) + self.x2h(x))
        return self.h2y(h), h

In [0]:
class MLP(nn.Module):
    def __init__(self, in_dim, out_dim, hid_dim, n_layer, act, use_bn, use_xavier, dropout):
        
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.hid_dim = hid_dim
        self.n_layer = n_layer
        self.use_bn = use_bn
        self.use_xavier = use_xavier
        self.dropout = dropout
        self.act = act
        
        if self.act == "relu": 
            self.act_fn = nn.ReLU()
        elif self.act == "tanh": 
            self.act_fn = nn.Tanh()
        elif self.act == "sigmoid": 
            self.act_fn = nn.sigmoid()
        else: 
            raise ValueError("Illegal activation function")
        
        self.layers = nn.ModuleList()
        
        self.layers.append(nn.Linear(in_dim, hid_dim))
        self.layers.append(self.act_fn)
        if self.use_bn:
            self.layers.append(nn.BatchNorm1d(hid_dim))
        self.layers.append(nn.Dropout(self.dropout))
        
        for i in range(n_layer-1):
            self.layers.append(nn.Linear(hid_dim, hid_dim))
            self.layers.append(self.act_fn)
            if self.use_bn:
                self.layers.append(nn.BatchNorm1d(hid_dim))
            self.layers.append(nn.Dropout(self.dropout))
        
        self.layers.append(nn.Linear(hid_dim, out_dim))
        
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

## Model Train

In [0]:
def train(net, partition, optimizer, criterion, args):
    trainloader = torch.utils.data.DataLoader(partition['train'], 
                                              batch_size=args.train_batch_size, 
                                              shuffle=True, num_workers=2)
    net.train()
    net.zero_grad()
    optimizer.zero_grad()

    correct = 0
    total = 0
    train_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.view(-1, 3072)
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

In [0]:
def validate(net, partition, criterion, args):
    valloader = torch.utils.data.DataLoader(partition['val'], 
                                            batch_size=args.valid_batch_size, 
                                            shuffle=False, num_workers=2)
    net.eval()
    net.zero_grad()
    optimizer.zero_grad()

    correct = 0
    total = 0
    val_loss = 0 
    with torch.no_grad():
        for data in valloader:
            images, labels = data
            images = images.view(-1, 3072)
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)

            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(valloader)
        val_acc = 100 * correct / total
    return val_loss, val_acc

In [0]:
def experiment(partition, args):
  
    x2h_net = MLP(args.in_dim, args.hidden_state_dim, args.x2h_hid_dim, args.x2h_n_layer, args.mlp_act, args.dropout, args.use_bn, args.use_xavier)
    h2h_net = MLP(args.hidden_state_dim, args.hidden_state_dim, args.h2h_hid_dim, args.h2h_n_layer, args.mlp_act, args.dropout, args.use_bn, args.use_xavier)
    h2y_net = MLP(args.hidden_state_dim, args.out_dim, args.h2y_hid_dim, args.h2y_n_layer, args.mlp_act, args.dropout, args.use_bn, args.use_xavier)
    net = RNN(args.train_batch_size, args.rnn_act, x2h_net, h2h_net, h2y_net)
    net.cuda()

    criterion = nn.CrossEntropyLoss()
    if args.optim == 'SGD':
        optimizer = optim.RMSprop(net.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(net.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(net.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    else:
        raise ValueError('In-valid optimizer choice')
    
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
        
    for epoch in range(args.epoch):  # loop over the dataset multiple times
        ts = time.time()
        net, train_loss, train_acc = train(net, partition, optimizer, criterion, args)
        val_loss, val_acc = validate(net, partition, criterion, args)
        te = time.time()
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        
        print('Epoch {}, Acc(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.2f}/{:2.2f}. Took {:2.2f} sec'.format(epoch, train_acc, val_acc, train_loss, val_loss, te-ts))
        
    result = {}
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    result['train_accs'] = train_accs
    result['val_accs'] = val_accs
    result['train_acc'] = train_acc
    result['val_acc'] = val_acc
    return vars(args), result

## Saving & Loading Result

In [0]:
import hashlib
import json
from os import listdir
from os.path import isfile, join
import pandas as pd

def save_exp_result(setting, result):
    exp_name = setting['exp_name']
    del setting['epoch']
    del setting['test_batch_size']

    hash_key = hashlib.sha1(str(setting).encode()).hexdigest()[:6]
    filename = './results/{}-{}.json'.format(exp_name, hash_key)
    result.update(setting)
    with open(filename, 'w') as f:
        json.dump(result, f)

    
def load_exp_result(exp_name):
    dir_path = './results'
    filenames = [f for f in listdir(dir_path) if isfile(join(dir_path, f)) if '.json' in f]
    list_result = []
    for filename in filenames:
        if exp_name in filename:
            with open(join(dir_path, filename), 'r') as infile:
                results = json.load(infile)
                list_result.append(results)
    df = pd.DataFrame(list_result) # .drop(columns=[])
    return df

## Visulaizations

In [0]:
def plot_acc(var1, var2, df):

    fig, ax = plt.subplots(1, 2)
    fig.set_size_inches(15, 6)
    sns.set_style("darkgrid", {"axes.facecolor": ".9"})

    sns.barplot(x=var1, y='train_acc', hue=var2, data=df, ax=ax[0])
    sns.barplot(x=var1, y='val_acc', hue=var2, data=df, ax=ax[1])
    
    ax[0].set_title('Train Accuracy')
    ax[1].set_title('Validation Accuracy')
    
    
def plot_loss_variation(var1, var2, df, **kwargs):

    list_v1 = df[var1].unique()
    list_v2 = df[var2].unique()
    list_data = []

    for value1 in list_v1:
        for value2 in list_v2:
            row = df.loc[df[var1]==value1]
            row = row.loc[df[var2]==value2]

            train_losses = list(row.train_losses)[0]
            val_losses = list(row.val_losses)[0]

            for epoch, train_loss in enumerate(train_losses):
                list_data.append({'type':'train', 'loss':train_loss, 'epoch':epoch, var1:value1, var2:value2})
            for epoch, val_loss in enumerate(val_losses):
                list_data.append({'type':'val', 'loss':val_loss, 'epoch':epoch, var1:value1, var2:value2})

    df = pd.DataFrame(list_data)
    g = sns.FacetGrid(df, row=var2, col=var1, hue='type', **kwargs)
    g = g.map(plt.plot, 'epoch', 'loss', marker='.')
    g.add_legend()
    g.fig.suptitle('Train loss vs Val loss')
    plt.subplots_adjust(top=0.89) # 만약 Title이 그래프랑 겹친다면 top 값을 조정해주면 됩니다! 함수 인자로 받으면 그래프마다 조절할 수 있겠죠?


def plot_acc_variation(var1, var2, df, **kwargs):
    list_v1 = df[var1].unique()
    list_v2 = df[var2].unique()
    list_data = []

    for value1 in list_v1:
        for value2 in list_v2:
            row = df.loc[df[var1]==value1]
            row = row.loc[df[var2]==value2]

            train_accs = list(row.train_accs)[0]
            val_accs = list(row.val_accs)[0]
            test_acc = list(row.test_acc)[0]

            for epoch, train_acc in enumerate(train_accs):
                list_data.append({'type':'train', 'Acc':train_acc, 'test_acc':test_acc, 'epoch':epoch, var1:value1, var2:value2})
            for epoch, val_acc in enumerate(val_accs):
                list_data.append({'type':'val', 'Acc':val_acc, 'test_acc':test_acc, 'epoch':epoch, var1:value1, var2:value2})

    df = pd.DataFrame(list_data)
    g = sns.FacetGrid(df, row=var2, col=var1, hue='type', **kwargs)
    g = g.map(plt.plot, 'epoch', 'Acc', marker='.')

    def show_acc(x, y, metric, **kwargs):
        plt.scatter(x, y, alpha=0.3, s=1)
        metric = "Valid Acc: {:1.3f}".format(list(metric.values)[0])
        plt.text(0.05, 0.95, metric,  horizontalalignment='left', verticalalignment='center', transform=plt.gca().transAxes, bbox=dict(facecolor='yellow', alpha=0.5, boxstyle="round,pad=0.1"))
    
    g = g.map(show_acc, 'epoch', 'Acc', 'val_acc')

    g.add_legend()
    g.fig.suptitle('Train Accuracy vs Val Accuracy')
    plt.subplots_adjust(top=0.89)

## Argument Define

In [0]:
import argparse

seeds = [123, 456, 789]
np.random.seed(seeds[0])
torch.manual_seed(seeds[0])

parser = argparse.ArgumentParser()
args = parser.parse_args("")

args.exp_name = 

# ====== model related ====== #
args.in_dim = 1024 #
args.out_dim = 1024 #
args.hidden_state_dim = 

args.hid_dim = 
args.n_layer = 

args.x2h_hid_dim = args.hid_dim
args.x2h_n_layer = args.n_layer
args.h2h_hid_dim = args.hid_dim
args.h2h_n_layer = args.n_layer
args.h2y_hid_dim = args.hid_dim
args.h2y_n_layer = args.n_layer

args.mlp_act = 
args.rnn_act = "tanh" #

# ====== optimizer related ====== #
args.lr = 
args.optim = 
args.epoch = 10 #

# ====== regularization related ====== #
args.dropout = 0.0
args.l2 = 0.0
args.use_bn = False
args.use_xavier = False

args.train_batch_size = 
args.test_batch_size = 

# ======  ====== #

def grid_hyperparameter_tuning(default_namespace, var1, vals1, var2, vals2):
    
    namespace = deepcopy(default_namespace)
    for val1 in vals1:
        for val2 in vals2:
            setattr(namespace, var1, val1)
            setattr(namespace, var2, val2)
            #print(namespace)
                
            setting, result = experiment(partition, deepcopy(namespace))
            save_exp_result(setting, result)
                
